In [11]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [12]:
import sqlite3
import numpy as np
from SlideRunner.dataAccess.database import Database
from tqdm import tqdm
from pathlib import Path
import openslide
import time
import pickle
import cv2
from Detection.data_loader import *

In [13]:
path = Path('/data/Datasets/EIPH_WSI/')

database = Database()
database.open(str(path/'EIPH.sqlite'))

size = 512
level = 1

files = []

In [14]:
getslides = """SELECT uid, filename FROM Slides"""
for currslide, filename in tqdm(database.execute(getslides).fetchall()):
    database.loadIntoMemory(currslide)

    check = True if 'erliner' in filename else False
    slidetype = 'Berliner Blau/' if check else 'Turnbull Blue/'

    slide_path = path / slidetype / filename

    slide = openslide.open_slide(str(slide_path))
    level = level#slide.level_count - 1
    level_dimension = slide.level_dimensions[level]
    down_factor = slide.level_downsamples[level]

    files.append(SlideContainer(slide_path,[[0], [1]], level, size, size))

  0%|          | 0/24 [00:00<?, ?it/s]

Loading DB into memory ...


  4%|▍         | 1/24 [00:00<00:06,  3.29it/s]

Loading DB into memory ...


  8%|▊         | 2/24 [00:01<00:13,  1.59it/s]

...

Loading DB into memory ...
Loading DB into memory ...
Loading DB into memory ...


 88%|████████▊ | 21/24 [00:05<00:00,  4.36it/s]

Loading DB into memory ...
Loading DB into memory ...


100%|██████████| 24/24 [00:06<00:00,  3.55it/s]

Loading DB into memory ...


In [15]:
result_regression = {}

model =  pickle.load(open("SVR_regression.p", "rb"))

In [16]:
for slide_container in tqdm(files):
    result_regression[slide_container.file.name] = []
    for x in range(0, slide_container.slide.level_dimensions[level][1] - 2 * size, int(size / 2)):
        for y in range(0, slide_container.slide.level_dimensions[level][0] - 2 * size, int(size / 2)):
            x_real = x  
            y_real = y  
            
            patch = slide_container.get_patch(x,y)
            histogram = np.concatenate((np.histogram(patch[:,:,0], bins=256)[0],
                                        np.histogram(patch[:,:,1], bins=256)[0],
                                        np.histogram(patch[:,:,2], bins=256)[0]))
            
            reg_pred = model.predict([histogram])[0]
            
            result_regression[slide_container.file.name].append(
                        np.array([x_real, y_real, x_real + size, y_real + size, reg_pred]))
            
    pickle.dump(result_regression, open("inference_result_regression_classic_SVR.p", "wb"))

100%|██████████| 24/24 [10:41<00:00, 27.00s/it]
